In [17]:
import numpy as np
import pandas as pd
import torch
import pickle

import sys
import os

sys.path.append(os.path.abspath(os.path.join('..', '..')))
from Functions_generation import generate_a_song_structure, sample_with_temp_topk, Char_RNN

In [18]:
with open("../../../Corpus/Encoding_RNN_LSTM/Char_level/encoding_map.pkl", "rb") as f:
    mapping = pickle.load(f)

mapping["PAD"] = len(mapping)
int2char = {i: ch for ch, i in mapping.items()}
nb_char = len(int2char)

In [19]:
matrix = pd.read_csv("../../../Markov/transition_matrix.csv")

states = np.array(matrix.iloc[6])
prob_transi = np.array(matrix.iloc[0:6])

In [20]:
vocab_size = len(int2char)
model = Char_RNN(vocab_size, num_layers=3)   

In [21]:
ckpt = torch.load("../Models/RNN_model.pt", map_location="cpu")

new_state_dict = {}
for key, value in ckpt["model_state_dict"].items():
    new_key = key.replace("_orig_mod.", "")           
    new_state_dict[new_key] = value

ckpt["model_state_dict"] = new_state_dict

model.load_state_dict(ckpt["model_state_dict"])

<All keys matched successfully>

I will give the same structure for RNN and LSTM to compare

In [22]:
struct = generate_a_song_structure(prob_transi.astype(float),states)

#struct = ['<BEGINNING>', '<COUPLET>', '<REFRAIN>', '<COUPLET>', '<REFRAIN>', '<END>']
struct = ['α', 'γ', 'ε', 'γ', 'ε', 'θ']

encoded = torch.tensor([mapping[c] for c in struct[1]], dtype=torch.long).unsqueeze(0)

['<BEGINNING>', '<INTRO>', '<COUPLET>', '<OUTRO>', '<END>']


Forbidden to generate

In [23]:
forbidden_generation = [i for i in range(64,72)]
[int2char[i] for i in forbidden_generation]

['α', 'β', 'γ', 'ε', 'ζ', 'η', 'θ', '€']

In [24]:
decod_structure = {"β" : "<INTRO>",
                   "γ" : "<COUPLET>",
                   "ε" : "<REFRAIN>",
                   "ζ" : "<PONT>",
                   "η" : "<OUTRO>",
                   "θ" : "<END>"}

In [25]:
context_gen = "voila pourquoi, "
context_encode = [mapping[i] for i in context_gen]
context_size = len(context_encode)

hid = model.init_hidden(batch_size=1)

for i in range(len(struct) - 2):

    print()
    print(decod_structure[struct[i+1]])

    if i == 0 : 
        context_encode.insert(0,mapping[struct[i+1]])
        context_encode.insert(1,mapping["\n"])
        encoded = torch.tensor(context_encode, dtype=torch.long).unsqueeze(0)

        print("".join([int2char[i] for i in context_encode[1:]]), end="")

        out, hid = model(encoded, hid)
        next_input = encoded[:, -1].unsqueeze(0)  # shape (1, 1)

    else : 
        #next_input = torch.tensor(mapping[struct[i+1]],dtype=torch.long).unsqueeze(0)
        next_input = torch.tensor(mapping[struct[i+1]],dtype=torch.long).view(1,1)

    last_input = 0
    first_input = True

    while last_input != 12:
        with torch.no_grad():
            out, hid = model(next_input, hid)   # out shape: (1, 1, vocab_size)

            out[0, -1][forbidden_generation] = float("-inf")

        # 4. On échantillonne un token
            next_token = sample_with_temp_topk(out[0, -1], temperature=0.8, top_k=20)
            last_input = next_token.item()

        # 5. Conditions d’impression
            if first_input and last_input == 12:  # Si début et le modèle sort directement <END>
                last_input = 0
                continue

            elif first_input and last_input == 0:  # Si début et sortie = \n
                first_input = False
                forbidden_generation.extend([0])
                print(int2char[next_token.item()], end="")

            elif next_token.item() != 12:
                first_input = False
                print(int2char[next_token.item()], end="")

                if 0 in forbidden_generation:
                    forbidden_generation.pop(-1)

        # 6. Préparer la prochaine entrée
            next_input = next_token.view(1, 1)

print("\n", decod_structure[struct[-1]])



<COUPLET>

voila pourquoi, ils se promettre
ça découpe c'est d'la musique ouais j'suis partout comme ça

<REFRAIN>
mon ombre
sur la banquette, j'ai vu la même maison, on parle d'accord dans la secte
un jour on s'en fout, ton rappeur s'embrouille, c'est mon seum on en ressente l'instru et ma vision cherche cette spectacle
comme si c'était la place du début d'la guerre aux blagues de toi
on baise très concentrée et j'vis ma force entièrement de la météan, le cash la course en ais-hop, faut qu'j'me dis pas vraiment des douze ans
j'suis dans le fond
pendant qu'j'me réveille, on fait pire
prends une mots, j'vois trop d'faux, j'suis grave sur nos chances de mon terminable
putain, tu vois  sku sku j'veux dire
tu vois  sku sku j'veux dire
tu vois c'que j'veux dire
tu vois  sku sku j'veux dire?
ok, slide, slide
ok, tu t'es baiser comme le mouvon
j'veux une bonne voix de mon pote, si on m'a dit qu'j'suis l'temps pour rentrer dans le piège si t'es un film d'un homme en moi
j'ai remplis la nuit
j